Importar librerias

In [ ]:
import requests
#Embeddings
from langchain_ollama import OllamaEmbeddings
#LLM
from langchain_ollama.llms import OllamaLLM
#Prompts
from langchain_core.prompts import ChatPromptTemplate
# document loaders
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader, TextLoader, Docx2txtLoader
# ChromaDB
from langchain_community.vectorstores import Chroma
# text splitter
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
#????
#from langchain_community.document_loaders import WebBaseLoader
#from langchain_text_splitters import RecursiveCharacterTextSplitter
#from bs4 import BeautifulSoup
#from langchain_core.runnables import RunnablePassthrough

# Preparar documentos

## Cargar docs

In [ ]:
# Cargar todos los PDFs de la carpeta "Contratos" con PyPDFLoader
loader = DirectoryLoader(
    path="./Contratos",
    glob="**/*.pdf",        # Cargar solo archivos PDF
    loader_cls=PyPDFLoader  # Cargar solo archivos PDF
)

documentos = loader.load()

## Chunking

In [ ]:

splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,       # caracteres por chunk
    chunk_overlap=50      # solapamiento entre chunks (mantiene contexto)
)

chunks = splitter.split_documents(documentos)

# Embeddings

In [ ]:
embeddings = OllamaEmbeddings(model="llama3.2")

# Chroma

## Crear base de datos vectorial

In [ ]:
vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory="./chroma_db"
)

## Cargar base vectorial

In [ ]:
# Cargar la base existente
vectorstore = Chroma(
    persist_directory="./chroma_db",
    embedding_function=embeddings
)

## Agregar nuevos documentos

In [ ]:
#vectorstore.add_documents(nuevos_chunks)

# RAG

In [ ]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

# LLM

### Templates para prompts (no son finales)

In [ ]:
tem_PagPrincipal = """
Eres un asistente especializado en contratos de arrendamiento. 
Tienes acceso a estas herramientas: 
    chat_analysis:      para consultas, resúmenes y análisis generales; 
    filter_search:      cuando el usuario quiere buscar contratos por criterios específicos como fecha, tipo, partes o ubicación; 
    generation_tool:    cuando el usuario quiere crear o sugerir cláusulas, generar borradores o redactar términos. 
Determina cuál usar según la solicitud y extrae los parámetros relevantes.
"""
tem_chat_analysis = """ 
Eres un asistente especializado en contratos de arrendamiento. 
Tu función es responder preguntas, proporcionar resúmenes y realizar análisis generales sobre los contratos de arrendamiento.
Utiliza la información disponible en los documentos para ofrecer respuestas precisas y detalladas a las consultas de los usuarios.
"""
tem_filter_search  = """
Eres un asistente especializado en buscar y filtrar contratos de arrendamiento.
Tu función es ayudar a los usuarios a encontrar contratos específicos basados en criterios como fecha, tipo de contrato, partes involucradas o ubicación.
"""
tem_generation_tool = """
Eres un asistente especializado en crear o sugerir cláusulas, generar borradores o redactar términos de contratos de arrendamiento.
Tu función es ayudar a los usuarios a generar contenido relacionado con contratos de arrendamiento, ya sea creando nuevas cláusulas, sugiriendo redacciones o generando borradores completos.
"""

## Tool Calling Agent 

## Pruebas